# Task 2 Conversational Demo

This notebook demonstrates **quick solutions** to transform our zero-shot pipeline into a more **conversational** system without changing the `facebook/bart-large-mnli` model.

## Overview
1. We'll initialize the same database and zero-shot analyzer.
2. We'll import our new **`ConversationalManager`** (from `src/conversational_manager`) to handle context windows.
3. We'll simulate a conversation, step by step, showing how we can pass the last `N` utterances as context.
4. We'll store results in the same `logs` table, so you can see how both Task 1 and Task 2 logs might coexist.

## Prerequisites
- Run `pip install -r requirements.txt`.
- Confirm your `config/model_config.ini` is configured as needed (e.g., `device=cpu` or `cuda`).


In [ ]:
import asyncio
import configparser
import json
import os
from IPython.display import display, Markdown

# We'll import the classes from our src/ package:
from src.db import Database
from src.logger import AsyncLogger
from src.sentiment_intent_analyzer import ZeroShotAnalyzer
from src.conversational_manager import ConversationalManager


## 1) Read configuration & initialize DB
We'll do the same steps as Task 1, but inside this notebook.

In [ ]:
config = configparser.ConfigParser()
config_path = os.path.join("..", "config", "model_config.ini")

if not os.path.exists(config_path):
    raise FileNotFoundError(f"model_config.ini not found at {config_path}")

config.read(config_path)

model_name = config["DEFAULT"].get("model_name", "facebook/bart-large-mnli")
device = config["DEFAULT"].get("device", "cpu")
db_url = config["DEFAULT"].get("db_url", "sqlite+aiosqlite:///./logs.db")

print("Model Name:", model_name)
print("Device:", device)
print("DB URL:", db_url)

## 2) Initialize Database & Analyzer

In [ ]:
db = Database(db_url)
await db.init_db()
logger = AsyncLogger(db)

analyzer = ZeroShotAnalyzer(
    model_name=model_name,
    device=device
)

## 3) Create our ConversationalManager
We'll set a small `context_window` of **2**.

In [ ]:
conv_manager = ConversationalManager(
    analyzer=analyzer,
    logger=logger,
    context_window=2
)

## 4) Load a conversation (the same or a different JSON) & simulate multi-turn
We'll re-use `conversation.json` from the parent `data` folder, or you can create a new custom conversation. Each turn we pass to `conv_manager.process_utterance`.

In [ ]:
data_path = os.path.join("..", "data", "conversation.json")

if not os.path.exists(data_path):
    raise FileNotFoundError(f"conversation.json not found at {data_path}")

with open(data_path, "r", encoding="utf-8") as f:
    conv_data = json.load(f)

conversation = conv_data.get("conversation", [])
print(f"Loaded {len(conversation)} utterances from conversation.json.")

### Let's process each utterance with context
We'll do it step by step and display the classification results.

> In a real conversational system, you'd do this **interactively** each time a new user message arrives.

In [ ]:
async def run_conversational_demo(conversation):
    results = []
    for turn in conversation:
        role = turn.get("role", "unknown")
        text = turn.get("text", "")

        if not text:
            continue

        # Process with context-based approach
        analysis = await conv_manager.process_utterance(role, text)
        results.append(analysis)

    return results

results = await run_conversational_demo(conversation)

print("Finished contextual classification!")
print(f"Logged {len(results)} messages in total.")

### 5) Inspect the results
We stored them in `logs.db` again, but let's just see them in-memory too.

In [ ]:
for idx, r in enumerate(results[:5]):
    display(Markdown(f"**Utterance {idx+1}:** Role: {r['role']} | Text: ...<truncated>...\n\n"
                    f"- Sentiment: {r['sentiment']}\n"
                    f"- Intent: {r['intent']}\n"))

## Conclusions
We've now demonstrated how the same **BART MNLI** zero-shot classifier can be turned into a **conversational** system by:

1. **Maintaining short memory** (context window) of previous utterances.
2. Combining them with the current text for classification.
3. Logging results to the same async DB.

>  This approach can be extended by adding rule-based or state-machine logic, session IDs, or other domain-specific logic.

**Thank you**